<a href="https://www.kaggle.com/code/xpertdl/class24-25-26?scriptVersionId=298007032" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import random
import numpy as np
from PIL import Image
from tqdm import tqdm


In [ ]:
cat_path = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat"
dog_path = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog"

In [ ]:
IMG_SIZE = 64
SAMPLES_PER_CLASS = 250
random.seed(42)

In [ ]:
classes = {
    cat_path: 0,
    dog_path: 1
}x

X = []
y = []

In [ ]:
for class_path, label in classes.items():
    all_images = os.listdir(class_path)
    random.shuffle(all_images)

    count = 0

    for img_name in tqdm(all_images, desc=f"Processing {class_path.split('/')[-1]}"):
        if count >= SAMPLES_PER_CLASS:
            break

        img_path = os.path.join(class_path, img_name)

        try:
            img = Image.open(img_path).convert("RGB")
            img = img.resize((IMG_SIZE, IMG_SIZE))

            img_array = np.array(img, dtype=np.float32) / 255.0

            X.append(img_array)
            y.append(label)

            count += 1

        except:
            continue

X = np.array(X)          
y = np.array(y)




In [ ]:
indices = np.arange(len(X))
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

In [ ]:
X = np.transpose(X, (0, 3, 1, 2))
y = y.reshape(-1, 1)

print("X shape:", X.shape)  
print("y shape:", y.shape)  


In [ ]:
import matplotlib.pyplot as plt


i = np.random.randint(0, len(X))
img = X[i].transpose(1, 2, 0)

plt.imshow(img)
plt.title("Dog" if y[i] == 1 else "Cat")
plt.axis("off")
plt.show()


In [ ]:

plt.figure(figsize=(10, 5))

for i in range(8):
    plt.subplot(2, 4, i + 1)

    img = X[i].transpose(1, 2, 0)
    plt.imshow(img)

    plt.title("Dog" if y[i] == 1 else "Cat")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [26]:
class ReLU:
  def __init__(self,):
    pass
  def forward(self, z):
    self.input_ = z
    return np.maximum(z, 0)

  def backward(self, delta):
    return delta * (self.input_ > 0).astype(int)


class Sigmoid:
  def __init__(self,):
    pass

  def forward(self, z):
    self.o =  1 / (1+ np.exp(-z))
    return self.o

  def backward(self, delta):
    return self.o * (1 - self. o)



def padding(x, pad):
    if pad == 0:
        return x
    return np.pad(x, ((0, 0), (pad, pad), (pad, pad)), mode="constant")



def BCELoss(y_hat, y):
  eps = 1e-6
  return - np.mean(y * np.log(y_hat +eps) + (1-y) * np.log(1 - y_hat + eps))


[Convolutional opetration](https://drive.google.com/file/d/1hZHd5K68KTeRgBdtOeMlSZgZzLr9-nfO/view?usp=sharing)

In [31]:
class Conv:
    def __init__(self, depth, kernels, kernel_size, stride=1, padding=0, activation="relu"):
        self.weight = np.random.randn(kernels, depth, kernel_size, kernel_size)
        self.bias = np.random.randn(kernels)

        
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        self.activation = ReLU()


    
    def forward(self, X):
        # X shape: (C, H, W)
        
        X = padding(X, self.padding)

        C, H, W = X.shape
        
        k = self.kernel_size
        s = self.stride
        
        out_channels = self.weight.shape[0]

        H_out = (H - k) // s + 1
        W_out = (W - k) // s + 1

        out = np.zeros((out_channels, H_out, W_out))
        

        for oc in range(out_channels):
            for i in range(H_out):
                for j in range(W_out):
                    h_start = i * s
                    w_start = j * s

                    region = X[:, h_start:h_start + k, w_start:w_start + k]

                    
                    out[oc, i, j] = np.sum(region * self.weight[oc]) + self.bias[oc]


        out = self.activation.forward(out)
        return out

    
    def backward(self, delta, lr):

        return delta

In [33]:
class MaxPool:
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
      
        C, H, W = X.shape
        k = self.kernel_size
        s = self.stride

        H_out = (H - k) // s + 1
        W_out = (W - k) // s + 1

        out = np.zeros((C, H_out, W_out))


        for c in range(C):
            for i in range(H_out):
                for j in range(W_out):
                    h_start = i * s
                    w_start = j * s
                    region = X[c, h_start:h_start+k, w_start:w_start+k]
                    out[c, i, j] = np.max(region)

        return out
    
    def backward(self, delta, lr):

        return delta

In [49]:
class Flatten:
    def __init__(self):
        pass

    def forward(self, X):
        self.input_shape = X.shape
        out =  X.flatten().reshape(1, -1)
        
        return out

    def backward(self, delta, lr):
        print("delta shape before: ", delta.shape)
        delta = delta.reshape(self.input_shape)
        print("delta shape After: ", delta.shape)

        return delta

In [50]:
class Dense:
  def __init__(self, features, out, activation = "ReLU", last_layer = False):
    self.weights = np.random.randn(out, features)
    self.bias = np.random.randn(1, out)
    self.last_layer = last_layer

    all_activation = {"ReLU": ReLU(), "Sigmoid": Sigmoid()}
    self.activation = all_activation[activation]


  def forward(self, X):
    self.input_ = X
    o = X @ self.weights.T + self.bias
    o = self.activation.forward(o)
    return o
    
  def backward(self, delta, lr):
    dl = delta

    if self.last_layer == False:
      dl = self.activation.backward(dl)


    dw = dl.T @ self.input_
    db = np.sum(dl, keepdims=True, axis=0)


    dX =  dl @ self.weights

    self.weights -= lr * dw
    self.bias -= lr * db

    return dX


In [51]:
class CNN:
    def __init__(self, layers):
        self.layers = layers

    def forward(self, X):
        x = X
        
        for layer in self.layers:
            x = layer.forward(x)

        return x

    def backward(self, delta, lr):
        dl = delta
        for layer in reversed(self.layers):
          dl = layer.backward(dl, lr)
        

In [52]:
model = CNN([
    Conv(depth=1, kernels=1, kernel_size=3, padding=0),
    MaxPool(2,2),
    Flatten(),
    Dense(4, 3),
    Dense(3, 1, activation="Sigmoid", last_layer = True),
])


In [17]:
X = np.random.randn(1,7,7)
y = np.array([[0]])

In [24]:
out = model.forward(X)
out

(1, 4)


array([[0.9738559]])

In [53]:
epochs =1
lr = 0.1

for i in range(epochs):
  y_hat = model.forward(X)
  loss = BCELoss(y_hat, y)

  n = X.shape[0]
  dl = (y_hat - y) / n


  model.backward(dl, lr)

  print(loss)




delta shape before:  (1, 4)
delta shape After:  (1, 2, 2)
0.9612265749191304
